In [30]:
import numpy as np
import json
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize,RegexpTokenizer
from sklearn.feature_extraction.text import CountVectorizer
import pickle
from sklearn.naive_bayes import MultinomialNB
from collections import Counter
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from nltk.sentiment.util import mark_negation
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

In [12]:

def load_pickle(filename):
    with open(filename,"rb") as f:
        dump=pickle.load(f)
    return dump

def dump_pickle(dump,filename):
    with open(filename,"wb") as f:
        pickle.dump(dump,f)


In [13]:
train_file='data/A1_Data/toy.json'
dev_file='data/A1_Data/toy.json'
tokenized_train="tokenized_train.pkl"
tokenized_dev="tokenized_dev.pkl"
log=10**5

In [14]:
train_file='data/A1_Data/train.json'
dev_file='data/A1_Data/dev.json'

In [15]:
def read_data(filename):
    x=[]
    y=[]
    
    file_reader=open(filename,"r")
    for line in file_reader:
        mapping=json.loads(line)
        x.append(mapping['review'])
        y.append(mapping['ratings'])
    
    return x,y

In [16]:
train_sent,train_y=read_data(train_file)
dev_sent,dev_y=read_data(dev_file)
print(len(train_sent),len(dev_sent))

1000000 200000


## Tokenizer

In [92]:
def tokenize_data(data):
    tokenized=[]
    for i in range(len(data)):
        if(i%100000==0):
            print(i)
        tokenized.append(word_tokenize(data[i]))
    return tokenized

In [15]:
# train_x=tokenize_data(train_sent)
# dev_x=tokenize_data(dev_sent)

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
0
100000


In [8]:
# print(len(train_x),train_x[0],type(train_x),tokenized_train)

In [9]:
# dump_pickle(train_x,tokenized_train)
# dump_pickle(dev_x,tokenized_dev)

In [152]:
word_tokenize("I am ANkesh. I am 21 :)")

['I', 'am', 'ANkesh', '.', 'I', 'am', '21', ':', ')']

## Loader

In [17]:
train_x=load_pickle(tokenized_train)
dev_x=load_pickle(tokenized_dev)

In [18]:
print(len(train_x),len(dev_x))

1000000 200000


In [49]:
stop_words = set(stopwords.words('english')) 
lemmatizer = WordNetLemmatizer() 
#Consider Stemming

In [50]:
def rem_stopwords_lemmatize(data):
    new_data=[]
    count=0
    for line in data:
        count+=1
        if(count%log==0):
            print(count)
        line=mark_negation(line,shallow=True,double_neg_flip=True)
        new_line=[]
        for word in line:
            lemmatized_word=lemmatizer.lemmatize(word)
            if lemmatized_word not in stop_words:
                new_line.append(lemmatized_word)
        new_data.append(new_line)
    return new_data

In [51]:
# train_x=rem_stopwords_lemmatize(train_x)
# dev_x=rem_stopwords_lemmatize(dev_x)

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
100000
200000


## Save Cleaned

In [52]:
# dump_pickle(train_x,'cleaned_train.pkl')
# dump_pickle(dev_x,'cleaned_dev.pkl')mean_squared_error(y_true, y_pred)

## Load Cleaned

In [19]:
train_x_comp=load_pickle('cleaned_train.pkl')
dev_x_comp=load_pickle('cleaned_dev.pkl')

In [161]:
# cut=50000
# train_x=train_x_comp[0:cut]
# train_y=train_y[0:cut]
# dev_x=dev_x_comp[0:cut]
# dev_y=dev_y[0:cut]

In [20]:
print(len(train_x_comp),len(dev_x_comp))
print(isinstance(train_x_comp[0][0],str))
for i in train_x_comp:
    for j in i:
        if(isinstance(j,str)==False):
            print(j)

1000000 200000
True


In [21]:
# Helper:https://www.ritchieng.com/machine-learning-multinomial-naive-bayes-vectorization/

## Prepare Data

In [33]:
def temp(x):
    return x

vect = CountVectorizer(analyzer=temp,ngram_range=(1, 2),max_df=0.5)
print(vect)
train_x_features = vect.fit_transform(train_x_comp)
dev_x_features = vect.transform(dev_x_comp)
train_x_features,dev_x_features

CountVectorizer(analyzer=<function temp at 0x7fe7cdd36e18>, binary=False,
        decode_error='strict', dtype=<class 'numpy.int64'>,
        encoding='utf-8', input='content', lowercase=True, max_df=0.5,
        max_features=None, min_df=1, ngram_range=(1, 2), preprocessor=None,
        stop_words=None, strip_accents=None,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, vocabulary=None)


(<1000000x663422 sparse matrix of type '<class 'numpy.int64'>'
 	with 61424664 stored elements in Compressed Sparse Row format>,
 <200000x663422 sparse matrix of type '<class 'numpy.int64'>'
 	with 12209492 stored elements in Compressed Sparse Row format>)

### TF-IDF

In [34]:
def temp(x):
    return x

vectorizer = TfidfVectorizer(analyzer=temp,ngram_range=(1,2))
vectorizer.fit(train_x_comp)
train_x_tfidf=vectorizer.transform(train_x_comp)
dev_x_tfidf=vectorizer.transform(dev_x_comp)
train_x_tfidf,dev_x_tfidf

In [23]:
def get_accuracy(gold,pred):
    length=len(gold)
    return np.sum(gold==pred)*1.0/length
def get_Fscore(gold,pred):
    return (f1_score(gold, pred, average='macro'),f1_score(gold, pred, average='micro'))      

## General Fitter

In [35]:
def GeneralFitter(X,Y,obj):
    %time obj.fit(X,Y)
    return obj

## Models

In [26]:
nb=MultinomialNB()
clf = LinearSVC(random_state=0, tol=1e-5,max_iter=1000)
log_reg = LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial', max_iter=100)
reg = LinearRegression()

In [27]:
def get_stats(X,Y,dev_X,dev_Y,obj):
    pred_X=obj.predict(X)
    pred_dev=obj.predict(dev_X)
    print(get_accuracy(Y,pred_X))
    print(get_accuracy(dev_y,pred_dev))
    print(get_Fscore(Y,pred_X))
    print(get_Fscore(dev_y,pred_dev))
    print(mean_squared_error(dev_y,pred_dev))
    print(confusion_matrix(Y, pred_X))
    print(confusion_matrix(dev_y, pred_dev))

## Feature filtering

In [133]:
selection_obj=SelectKBest(chi2, k=1000)
X_new = selection_obj.fit_transform(train_x_features, train_y)
dev_new=selection_obj.transform(dev_x_features)
X_new,dev_new

In [36]:
nb_cv=MultinomialNB()
GeneralFitter(train_x_features,train_y,nb_cv)
get_stats(train_x_features,train_y,dev_x_features,dev_y,nb_cv)

CPU times: user 2.19 s, sys: 216 ms, total: 2.41 s
Wall time: 1.28 s
0.651809
0.613925
(0.5790082449946594, 0.651809)
(0.5249208163419372, 0.613925)
1.0594
[[112653  22560   7604   3551   3792]
 [ 20777  29410  19249   8788   3046]
 [ 10846   8410  48371  34821   8133]
 [  8539   3915  15745 133360  58069]
 [ 19802   2389   7104  81051 328015]]
[[21894  4700  1567   794   834]
 [ 4413  4439  4715  1868   704]
 [ 2263  2147  7518  8432  1756]
 [ 1791   758  3929 24605 12923]
 [ 4242   457  1307 17615 64329]]


In [38]:
nb_tf=MultinomialNB()
GeneralFitter(train_x_tfidf,train_y,nb_tf)
get_stats(train_x_tfidf,train_y,dev_x_tfidf,dev_y,nb_tf)

CPU times: user 2.18 s, sys: 68.3 ms, total: 2.25 s
Wall time: 1.16 s
0.576402
0.559105
(0.34420393231615176, 0.576402)
(0.3235664606041289, 0.559105)
1.826435
[[103870      8     94   7240  38948]
 [ 20713    120    345  19534  40558]
 [  7522      3    831  35947  66278]
 [  2592      9     27  42910 174090]
 [  2644     17     65   6964 428671]]
[[19714     5    20  1455  8595]
 [ 3951     4    56  3776  8352]
 [ 1489     3    66  6855 13703]
 [  511     0    10  6342 37143]
 [  577     1     4  1673 85695]]


In [39]:
log_reg_cv = LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial', max_iter=100)
GeneralFitter(train_x_features,train_y,log_reg_cv)
get_stats(train_x_features,train_y,dev_x_features,dev_y,log_reg_cv)

KeyboardInterrupt: 

AttributeError: 'list' object has no attribute 'shape'

In [ ]:
log_reg_tf=MultinomialNB()
GeneralFitter(train_x_tfidf,train_y,log_reg_tf)
get_stats(train_x_tfidf,train_y,dev_x_tfidf,dev_y,log_reg_tf)